In [1]:
import numpy as np
import pandas as pd

from connection import connect

# Fase de exploración

In [2]:
inspector_co, etl_con, co_sa = connect()

Esquemas encontrados: ['hr', 'person', 'production', 'public', 'purchasing', 'sales']

--- Tablas de Negocio Encontradas ---
Esquema 'hr' (6 tablas):
  > ['department', 'employee', 'employee_department_history', 'employee_pay_history', 'job_candidate']...
Esquema 'person' (13 tablas):
  > ['business_entity', 'address', 'address_type', 'business_entity_address', 'business_entity_contact']...
Esquema 'production' (25 tablas):
  > ['illustration', 'bill_of_materials', 'culture', 'document', 'location']...
Esquema 'public' (4 tablas):
  > ['awbuild_version', 'database_log', 'error_log', 'sysdiagrams']...
Esquema 'purchasing' (5 tablas):
  > ['product_vendor', 'purchase_order_detail', 'purchase_order_header', 'ship_method', 'vendor']...
Esquema 'sales' (19 tablas):
  > ['country_region_currency', 'credit_card', 'customer', 'currency', 'currency_rate']...


In [6]:
inspector_co.get_columns('person', schema='person')

[{'name': 'business_entity_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'person_type',
  'type': CHAR(length=2),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'name_style',
  'type': BOOLEAN(),
  'nullable': False,
  'default': 'false',
  'autoincrement': False,
  'comment': None},
 {'name': 'title',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'first_name',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'middle_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'last_name',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'suffix',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoi

In [8]:
# Lista de tablas que queremos inspeccionar (ajusta nombres si tu esquema cambia)
candidate_tables = [
    ('person', 'person'),
    ('sales', 'customer'),
    ('person', 'address'),
    ('person', 'state_province'),
    ('person', 'email_address'),
    ('person', 'person_phone'),
    ('sales', 'sales_territory'),
    ('sales', 'sales_order_header')
]

for schema, table in candidate_tables:
    try:
        cols = inspector_co.get_columns(table, schema=schema)
        print(f"\n{schema}.{table} -> {len(cols)} columnas")
        for c in cols:
            print(f"  - {c['name']:30} {c['type']}")
    except Exception as e:
        print(f"\nNo se pudo obtener columnas de {schema}.{table}: {e}")



person.person -> 13 columnas
  - business_entity_id             INTEGER
  - person_type                    CHAR(2)
  - name_style                     BOOLEAN
  - title                          TEXT
  - first_name                     TEXT
  - middle_name                    TEXT
  - last_name                      TEXT
  - suffix                         TEXT
  - email_promotion                INTEGER
  - additional_contact_info        TEXT
  - demographics                   TEXT
  - rowguid                        CHAR(36)
  - modified_date                  TIMESTAMP

sales.customer -> 7 columnas
  - customer_id                    INTEGER
  - person_id                      INTEGER
  - store_id                       INTEGER
  - territory_id                   INTEGER
  - account_number                 TEXT
  - rowguid                        CHAR(36)
  - modified_date                  TIMESTAMP

person.address -> 9 columnas
  - address_id                     INTEGER
  - address_line_1       

Consulta de union de valores principales


    SELECT
        c.account_number as alternate_key,
        p.title,
        p.first_name,
        p.middle_name,
        p.last_name,
        p.suffix,
        p.demographics,
        e.email_address,
        pp.phone_number,
        a.address_line_1,
        a.address_line_2
    FROM sales.customer c
    INNER JOIN person.person p
        ON c.person_id = p.business_entity_id
    INNER JOIN person.email_address e
        ON p.business_entity_id = e.business_entity_id
    INNER JOIN person.person_phone pp
        ON p.business_entity_id = pp.business_entity_id
    INNER JOIN person.business_entity_address bea
        ON p.business_entity_id = bea.business_entity_id
    INNER JOIN person.address a
        ON bea.address_id = a.address_id

